# Calculate Detector Counts: NEI
Compute detector counts, including non-equilibrium ionization.

In [30]:
import os
import glob

import numpy as np
import h5py
import astropy.units as u
import distributed

import synthesizAR
from synthesizAR.instruments import InstrumentSDOAIA
from synthesizAR.atomic import EmissionModel

In [47]:
client = distributed.Client()
client

Client Scheduler: tcp://127.0.0.1:34799 Dashboard: http://127.0.0.1:8787,Cluster Workers: 64 Cores: 64 Memory: 270.38 GB


In [2]:
# Load field
field = synthesizAR.Skeleton.restore('/storage-home/w/wtb2/data/timelag_synthesis/high_frequency/field_checkpoint/')

No HMI fits file supplied. A new HMI map object will not be created.
/storage-home/w/wtb2/anaconda3/envs/synthesizar/lib/python3.6/site-packages/synthesizAR-0.1.dev528-py3.6.egg/synthesizAR/field.py:187: SunpyDeprecationWarning: This property is only valid for non-rotated WCS
  lcx, rcx = self.hmi_map.xrange + self.hmi_map.scale.axis1*u.Quantity([boundary_clipping[0], -boundary_clipping[0]], u.pixel)
/storage-home/w/wtb2/anaconda3/envs/synthesizar/lib/python3.6/site-packages/synthesizAR-0.1.dev528-py3.6.egg/synthesizAR/field.py:188: SunpyDeprecationWarning: This property is only valid for non-rotated WCS
  lcy, rcy = self.hmi_map.yrange + self.hmi_map.scale.axis2*u.Quantity([boundary_clipping[1], -boundary_clipping[1]], u.pixel)
/storage-home/w/wtb2/anaconda3/envs/synthesizar/lib/python3.6/site-packages/synthesizAR-0.1.dev528-py3.6.egg/synthesizAR/field.py:195: SunpyDeprecationWarning: This property is only valid for non-rotated WCS
  bbox = np.array([self._convert_angle_to_length(self

In [3]:
# Load emission model
em_model = EmissionModel.restore('/storage-home/w/wtb2/data/timelag_synthesis/high_frequency/nei/emission_model_dominant.json')

In [4]:
# Build instrument and observer
aia = InstrumentSDOAIA([0, 30000]*u.s, use_temperature_response_functions=False)
observer = synthesizAR.Observer(field, [aia], parallel=True)

In [5]:
observer.build_detector_files('/storage-home/w/wtb2/data/timelag_synthesis/high_frequency/nei/',
                              ds=field._convert_angle_to_length(1.0*u.arcsec))

/storage-home/w/wtb2/anaconda3/envs/synthesizar/lib/python3.6/site-packages/scipy/interpolate/_fitpack_impl.py:299: RuntimeWarning: The maximal number of iterations (20) allowed for finding smoothing
spline with fp=s has been reached. Probable cause: s too small.
(abs(fp-s)/s>0.001)
  warnings.warn(RuntimeWarning(_iermess[ier][0]))
/storage-home/w/wtb2/anaconda3/envs/synthesizar/lib/python3.6/site-packages/scipy/interpolate/_fitpack_impl.py:299: RuntimeWarning: A theoretically impossible result when finding a smoothing spline
with fp = s. Probable cause: s too small. (abs(fp-s)/s>0.001)
  warnings.warn(RuntimeWarning(_iermess[ier][0]))
/storage-home/w/wtb2/anaconda3/envs/synthesizar/lib/python3.6/site-packages/synthesizAR-0.1.dev528-py3.6.egg/synthesizAR/instruments/base.py:116: SunpyDeprecationWarning: This property is only valid for non-rotated WCS
  delta_x = np.fabs(field.clipped_hmi_map.xrange[1] - field.clipped_hmi_map.xrange[0])
/storage-home/w/wtb2/anaconda3/envs/synthesizar/li

In [ ]:
tasks = observer.flatten_detector_counts(emission_model=em_model)

In [ ]:
tasks['SDO_AIA_parameters'].compute()

In [ ]:
tasks['SDO_AIA_counts'].compute()

Instead of running these in parallel, find the ones that didn't execute and run them by hand...

In [38]:
for channel in aia.channels:
    print(channel['name'])
    completed_loops = [os.path.basename(g).split('.')[0] 
                       for g in glob.glob(os.path.join(aia.tmp_file_template.format(channel['name']),'*.npy'))]
    incomplete_loops = [loop for loop in field.loops if loop.name not in completed_loops]
    flat_emiss = aia.flatten_emissivities(channel,em_model)
    for loop in incomplete_loops:
        print(loop.name)
        counts = aia.calculate_counts_full(channel,loop,em_model,flat_emiss)
        loop_num = field.loops.index(loop)
        loop_path = os.path.join(aia.tmp_file_template.format(channel['name']),f'{loop.name}.npy')
        aia.interpolate_and_store(counts,loop,aia.observing_time,observer._interpolated_loop_coordinates[loop_num],
                                  save_path=loop_path)

94
loop000412
loop000420
loop001019
loop001066
loop002397
131
loop001050
171
loop000368
loop000767
loop001255
loop002327
193
loop001152
loop001741
loop001952
211
loop000753
loop001912
loop004054
loop004674
335
loop000355
loop002114
loop002190
loop003269


And now merge into one big file by hand...

In [44]:
procedures = {}
for channel in aia.channels:
    tmp = []
    for loop in field.loops:
        tmp.append((os.path.join(aia.tmp_file_template.format(channel['name']),f'{loop.name}.npy'),
                    counts.unit.to_string()))
    procedures[channel['name']] = tmp

In [46]:
observer.assemble_arrays(procedures,aia.counts_file)

Now, compute the maps

In [48]:
tasks = observer.bin_detector_counts('/storage-home/w/wtb2/data/timelag_synthesis/high_frequency/nei/')

In [49]:
jobs = client.compute(tasks['SDO_AIA'])

In [50]:
jobs=None